In [ ]:
# Here we are using only the Dense Layer

# Control the output value using Relu Activation function and Predict the output by sigmoid function
# input_dim is the value 0f max lenth of padded sequence

# Step 1: Convert the Courpus data into Integer encoding
# Step 2: Build the Keras model using only Dense Layer

In [2]:
import pandas as pd

# Corpus 
positive = ['This is an excellent movie','The move was fantastic I like it',
            'You should watch it is brilliant','Exceptionally good','Wonderfully directed and executed I like it',
             'Its a fantastic series','Never watched such a brillent movie','It is a Wonderful movie']

negative = ["horrible acting",'waste of money','pathetic picture','It was very boring',
            'I did not like the movie','The movie was horrible','I will not recommend',
            'The acting is pathetic']

In [4]:
reviews = [(pr,"positive") for pr in positive] + [(nr,"negative") for nr in negative]

data = pd.DataFrame(reviews,columns=["comment",'review'])

# shuffle the DataFrame rows
data = data.sample(frac = 1)

data.head(3)

,comment,review
15,The acting is pathetic,negative
3,Exceptionally good,positive
12,I did not like the movie,negative


In [7]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
data['review'] = lb_make.fit_transform(data["review"])

data.head(2)

,comment,review
15,The acting is pathetic,0
3,Exceptionally good,1


In [8]:
# lower the text
data['comment'] = data['comment'].str.lower()

# find max_length sequence

max_seq_len = max([len([y for y in x.split()]) for x in data['comment']])
max_seq_len

7

In [9]:
# number of samples
num_samples = data.shape[0]
num_samples

16

In [10]:
from keras.preprocessing import text

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(data['comment'])
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in data['comment']]

Using TensorFlow backend.


In [11]:
vocab_size = len(id2word)

vocab_size

44

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_inputs = pad_sequences(wids,padding='post')

padded_inputs

array([[ 3,  9,  4, 10,  0,  0,  0],
       [14, 15,  0,  0,  0,  0,  0],
       [ 5, 16, 11,  6,  3,  1,  0],
       [ 2,  4,  7, 17,  1,  0,  0],
       [18, 19, 20,  2,  4, 21,  0],
       [22,  4, 23, 24,  1,  0,  0],
       [25, 26, 27,  0,  0,  0,  0],
       [28,  7, 12, 29,  0,  0,  0],
       [ 3,  1,  8, 13,  0,  0,  0],
       [ 5, 30, 11, 31,  0,  0,  0],
       [13,  9,  0,  0,  0,  0,  0],
       [32, 33, 34, 35,  5,  6,  2],
       [36, 37, 38,  7, 39,  1,  0],
       [ 2,  8, 40, 41,  0,  0,  0],
       [10, 42,  0,  0,  0,  0,  0],
       [ 3, 43,  8, 12,  5,  6,  2]])

In [13]:
y = data['review'].values # output labels

y

array([0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1], dtype=int64)

In [17]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_dim=7, activation='relu'))  # max_seq_len = 7 (Features)
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(padded_inputs, y, epochs=150, batch_size=10)
# evaluate the keras model


Epoch 1/150
16/16 [==============================] - 0s 16ms/step - loss: 3.0034 - accuracy: 0.6250
Epoch 2/150
16/16 [==============================] - 0s 370us/step - loss: 2.8440 - accuracy: 0.6250
Epoch 3/150
16/16 [==============================] - 0s 310us/step - loss: 2.6875 - accuracy: 0.6250
Epoch 4/150
16/16 [==============================] - 0s 450us/step - loss: 2.5344 - accuracy: 0.6250
Epoch 5/150
16/16 [==============================] - 0s 338us/step - loss: 2.3729 - accuracy: 0.6250
Epoch 6/150
16/16 [==============================] - 0s 137us/step - loss: 2.2377 - accuracy: 0.6250
Epoch 7/150
16/16 [==============================] - 0s 0us/step - loss: 2.0880 - accuracy: 0.6250
Epoch 8/150
16/16 [==============================] - 0s 249us/step - loss: 1.9637 - accuracy: 0.6250
Epoch 9/150
16/16 [==============================] - 0s 321us/step - loss: 1.8397 - accuracy: 0.6250
Epoch 10/150
16/16 [==============================] - 0s 62us/step - loss: 1.6907 - accuracy:

In [59]:
def predict_Sentiment(text):
    
    # lower the text
    text = text.lower()
    
    # Split the text
    token_text = text.split()
    
    # Convert to word to int
    word_2id = [word2id[x] for x in token_text]
    
    # pad the sequence
    
    padded_inputs = pad_sequences([word_2id],maxlen=7)
    
    result = model.predict(padded_inputs)[0][0]
    
    if result>0.5:
        print("Review: Positive")
    else:
        print("Review: Negative")


In [65]:
text = 'good'

predict_Sentiment(text)

Review: Positive


In [67]:
# Bad its new word 
# This is will not work for the words which are not part of the actual corpus data

text = 'bad'

predict_Sentiment(text)

KeyError: 'bad'